In [ ]:
# default_exp m5

In [ ]:
# hide
import sys

sys.path.append("..")
import pandas as pd

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
# hide
from nbdev.showdoc import *

# Data M5

> Getting the M5 data into a timerelevant dataloader.



In [ ]:
# export
from fastcore.all import *
from fastcore.imports import *
from fastai2.basics import *
from fastai2.data.transforms import *
from fastai2.tabular.core import *
from fastseq.data.all import *
from fastseq.core import *
from fastseq.data.all import *

In [ ]:
path = Path('../data/m5')
path.ls()

(#11) [Path('../data/m5/sales_train_validation.csv'),Path('../data/m5/M5-Competitors-Guide_Final-1.pdf'),Path('../data/m5/sample_submission.csv'),Path('../data/m5/Screenshot from 2020-03-04 13-24-55.png'),Path('../data/m5/calendar.csv'),Path('../data/m5/prices'),Path('../data/m5/rows'),Path('../data/m5/Screenshot from 2020-03-04 13-25-42.png'),Path('../data/m5/rows2'),Path('../data/m5/calender_d.csv')...]

In [ ]:
df_cal = pd.read_csv(path / 'calendar.csv')
df_cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
df_cal = df_cal[[c for c in list(df_cal.columns) if ('event' not in c)]]
df_cal.index = df_cal['d']
df_cal = df_cal.T
df_cal

d,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
date,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,2011-02-03,2011-02-04,2011-02-05,2011-02-06,2011-02-07,...,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
wm_yr_wk,11101,11101,11101,11101,11101,11101,11101,11102,11102,11102,...,11619,11620,11620,11620,11620,11620,11620,11620,11621,11621
weekday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,...,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
wday,1,2,3,4,5,6,7,1,2,3,...,7,1,2,3,4,5,6,7,1,2
month,1,1,1,2,2,2,2,2,2,2,...,6,6,6,6,6,6,6,6,6,6
year,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,...,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016
d,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
snap_CA,0,0,0,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
snap_TX,0,0,0,1,0,1,0,1,1,1,...,0,1,1,1,0,1,0,0,0,0
snap_WI,0,0,0,0,1,1,0,1,1,0,...,0,1,1,0,1,1,0,0,0,0


In [ ]:
df_cal.T.to_csv(path / 'calender_d.csv')

## price data

In [ ]:
df_price = pd.read_csv(path / 'sell_prices.csv')
df_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
# export
def add_days(df_sales, days = 28*2, fill = -1):
    for i in range(days):
        df_sales['d_'+str(i+1914)] = fill
    return df_sales

In [ ]:
# export
@delegates(pd.Series.interpolate)
def interpolate_w_starting_nan(s, **kwargs):
    if s.iloc[0] != s.iloc[0]:
        for i,o in enumerate(s != s):
            if not o:
                s.iloc[0] = s.iloc[i]
                break
    return s.interpolate(**kwargs)

In [ ]:
s = pd.Series([np.nan,np.nan,np.nan, 0, 2, np.nan, 6])
s = interpolate_w_starting_nan(s)
test_eq((s!=s).sum(),0)
s = pd.Series([np.nan,np.nan,np.nan, 0, 1, np.nan, 3], index = ['d'+str(i) for i in range(7)])
s = interpolate_w_starting_nan(s)
test_eq((s!=s).sum(),0)
s

d0    0.0
d1    0.0
d2    0.0
d3    0.0
d4    1.0
d5    2.0
d6    3.0
dtype: float64

In [ ]:
df_cal = pd.read_csv(path / 'calender_d.csv')
df_cal.index = df_cal['d']
d2wk = dict(df_cal.T.loc['wm_yr_wk',:])
del df_cal
pd.Series(d2wk)

d_1       11101
d_2       11101
d_3       11101
d_4       11101
d_5       11101
          ...  
d_1965    11620
d_1966    11620
d_1967    11620
d_1968    11621
d_1969    11621
Length: 1969, dtype: int64

In [ ]:
# export
def get_prices(df_price, store_id, item_id, method = 'linear'):    
    _df = df_price[df_price['store_id'] == store_id]
    _df = _df[_df['item_id'] == item_id]
    _df['wm_yr_wk'] = _df['wm_yr_wk'].astype(int)
    _df.index = _df['wm_yr_wk']
    p = dict(_df['sell_price'])
    wm_yr_wk = pd.Series({i:p[i] if i in p else np.nan for i in range(11100,11622)})
    wm_yr_wk = interpolate_w_starting_nan(wm_yr_wk, method = method)
    return wm_yr_wk

In [ ]:
p = get_prices(df_price, 'CA_1', 'HOBBIES_1_001')
p

NameError: name 'df_price' is not defined

In [ ]:
l = L()
for i in range(1,1970):
    l.append(p[d2wk['d_'+str(i)]])    
pd.Series(l)

0       9.58
1       9.58
2       9.58
3       9.58
4       9.58
        ... 
1964    8.38
1965    8.38
1966    8.38
1967    8.38
1968    8.38
Length: 1969, dtype: float64

In [ ]:
# export
def _get_price_s(row, df_price):    
    p = get_prices(df_price, row['store_id'], row['item_id'])
    dct = {'store_id':row['store_id'],'item_id':row['item_id']}
#     for i in range(1,1970):
    dct.update({'d_'+str(i): p[d2wk['d_'+str(i)]] for i in range(1,1970)})
    return dct

In [ ]:
df_price = pd.read_csv(path / 'sell_prices.csv')

In [ ]:
# (path / 'prices').mkdir()
# for j, store_id in enumerate(set(df_price['store_id'])):
#     _df = df_price[df_price['store_id'] == store_id]
#     for i, item_id in enumerate(set(_df['item_id'])):
#         df_item = _df[_df['item_id'] == item_id]        
#         p = get_prices(df_item, store_id, item_id)
#         dct = {'store_id':store_id,'item_id':item_id}
#         for i in range(1,1970):
#             dct['d_'+str(i)] = p[d2wk['d_'+str(i)]]  
#         json.dump(dct, open(path / 'prices' / str(store_id+'_'+item_id +'.csv'),'w'))   
#         if i % 100 == 0:
#             print(f"{j}/10\t{i}/3049")

In [ ]:
# export
import csv
def csv_gen(file):
    with open(file, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        for row in reader:
            yield {k:v for k,v in zip(header,row)}

In [ ]:
for o in csv_gen(path / 'calender_d.csv') :
    test_eq(type(o), dict)
    test_eq(o.keys(),{'d': 'd_1', 'date': '2011-01-29',
                     'wm_yr_wk': '11101', 'weekday': 'Saturday',
                     'wday': '1', 'month': '1', 'year': '2011',
                     'snap_CA': '0', 'snap_TX': '0', 'snap_WI': '0'}.keys())
 

In [ ]:
import os
if not os.path.isfile(path / 'prices.csv'):
    from multiprocessing.dummy import Pool as ThreadPool
    pool = ThreadPool(10)
    df_price = pd.read_csv(path / 'sell_prices.csv')
    prices = pool.map(_get_price_s,  (csv_gen(path / 'sales_train_validation.csv'),
                                      df_price))
    prices = pd.DataFrame(prices)
    prices.to_csv(path / 'prices.csv',index=False)
prices = pd.read_csv(path / 'prices.csv')
prices.head()

TypeError: _get_price_s() missing 1 required positional argument: 'df_price'

## M5

In [ ]:
df_cal = pd.read_csv(path / 'calender_d.csv')
df_cal.index = df_cal['d']
del df_cal['d']
df_cal

,date,wm_yr_wk,weekday,wday,month,year,d.1,snap_CA,snap_TX,snap_WI
d,,,,,,,,,,
d_1,2011-01-29,11101,Saturday,1,1,2011,d_1,0,0,0
d_2,2011-01-30,11101,Sunday,2,1,2011,d_2,0,0,0
d_3,2011-01-31,11101,Monday,3,1,2011,d_3,0,0,0
d_4,2011-02-01,11101,Tuesday,4,2,2011,d_4,1,1,0
d_5,2011-02-02,11101,Wednesday,5,2,2011,d_5,1,0,1
...,...,...,...,...,...,...,...,...,...,...
d_1965,2016-06-15,11620,Wednesday,5,6,2016,d_1965,0,1,1
d_1966,2016-06-16,11620,Thursday,6,6,2016,d_1966,0,0,0
d_1967,2016-06-17,11620,Friday,7,6,2016,d_1967,0,0,0


In [ ]:
# export
def _slice_series(dct, s_slice=None, add_zeros = 28*2):
    s_slice = ifnone(s_slice, slice(6,None))
    return [float(o) for k,o in dct.items() if ('d_' in k or 'F' in k)]+[0]*add_zeros
# @delegates(_to_series)
def to_contained_series(dct, series_column_name = 'sales', **kwargs):    
    data={k:v for k,v in dct.items() if ('d_' not in k and 'F' not in k)}
    data[series_column_name] = _slice_series(dct, **kwargs)
    return data

In [ ]:
import orjson
def _get_prices_dct(row):
    return orjson.loads(open(path / 'prices' / str(row['item_id']+'_'+row['store_id'] +'.json'),'r').read())

def _do_row(sales):
    s = to_contained_series(sales)
    for c in df_cal.columns:
        if c is not 'd.1':
            s[c] = list(df_cal[c])
    p = _get_prices_dct(s)
    f = p.pop('item_id') + '_'
    f += p.pop('store_id')
    s['prices'] = [o for k,o in p.items()] 
    return save_row(s, path / 'rows', fname= f)

In [ ]:
o = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
len(o), 28*2

(57, 56)

In [ ]:
for i, sales in enumerate(csv_gen(path / 'sales_train_validation.csv')):
    r = _do_row(sales)
    break
r

JSONDecodeError: Expecting ',' delimiter: line 20372 column 36 (char 706135)

In [ ]:
import time

In [ ]:
%%time
st = time.time()
for i, sales in enumerate(csv_gen(path / 'sales_train_validation.csv')):
    r = _do_row(sales)
    if i%100 == 0 and i != 0:  
        d = time.time()-st
        print(f"{i} / 30 000 ",np.round(d),np.round(((d/i)*30000)))

100 / 30 000  3.0 981.0
200 / 30 000  6.0 961.0
300 / 30 000  10.0 956.0
400 / 30 000  13.0 964.0
500 / 30 000  16.0 974.0
600 / 30 000  20.0 980.0
700 / 30 000  23.0 1000.0
800 / 30 000  27.0 1002.0
900 / 30 000  30.0 1003.0
1000 / 30 000  33.0 1004.0
1100 / 30 000  37.0 1015.0
1200 / 30 000  41.0 1016.0
1300 / 30 000  44.0 1021.0
1400 / 30 000  48.0 1025.0
1500 / 30 000  51.0 1028.0
1600 / 30 000  55.0 1031.0
1700 / 30 000  59.0 1036.0
1800 / 30 000  62.0 1039.0
1900 / 30 000  66.0 1043.0
2000 / 30 000  70.0 1048.0
2100 / 30 000  74.0 1053.0
2200 / 30 000  77.0 1056.0
2300 / 30 000  81.0 1060.0
2400 / 30 000  85.0 1064.0
2500 / 30 000  89.0 1070.0
2600 / 30 000  93.0 1077.0
2700 / 30 000  98.0 1084.0
2800 / 30 000  102.0 1093.0
2900 / 30 000  107.0 1103.0
3000 / 30 000  111.0 1111.0
3100 / 30 000  116.0 1120.0
3200 / 30 000  120.0 1126.0
3300 / 30 000  124.0 1131.0
3400 / 30 000  129.0 1136.0
3500 / 30 000  133.0 1142.0
3600 / 30 000  138.0 1147.0
3700 / 30 000  142.0 1153.0
3800 / 3

In [ ]:
def _copy(self, target):
    import shutil
    assert self.is_file()
    shutil.copy(str(self), str(target))  # str() only there for Python < (3, 6)

Path.copy = _copy

In [ ]:
for i, f in enumerate(Path('../data/m5/rows').glob('*.json')):
    ts = get_ts_datapoint(f)
    assert np.mean(ts['ts_con']['sales'][-(28*2):]) == 0.
    if i == 4000:
        break

In [ ]:
list(Path('../data/m5/rows').glob('*.json'))[0]

Path('../data/m5/rows/FOODS_2_155_WI_2.json')

In [ ]:
# f = list(Path('../data/m5/rows').glob('.ts_meta'))[0]
# f.copy(str(f).replace('m5','m5_tiny'))
n = 0

for f in Path('../data/m5/rows').glob('FOODS_2_1*_CA_3.json'):
    f.copy(str(f).replace('m5','m5_tiny'))
    n+=1    
for f in Path('../data/m5/rows').glob('FOODS_2_1*_TX_3.json'):
    f.copy(str(f).replace('m5','m5_tiny'))
    n+=1   
for f in Path('../data/m5/rows').glob('HOUSEHOLD_2_1*_WI_3.json'):
    f.copy(str(f).replace('m5','m5_tiny'))
    n+=1   
n

299

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 03_m5.ipynb.
Converted 050_nbeats.models_multi.ipynb.
Converted 05_nbeats.models.ipynb.
Converted 06_nbeats.callbacks.ipynb.
Converted 07_nbeats.learner.ipynb.
Converted 08_nbeats.interpret.ipynb.
Converted 11_metrics.ipynb.
Converted 12_compare.ipynb.
Converted index.ipynb.
Converted tab.model.ipynb.
